In [2]:
import requests
from requests import Request, Session
import json
import pandas as pd
import time
from datetime import datetime
import numpy as np

with open('market_cap_id.json','r') as r:
    data = json.load(r)
    filter_sym_id = data['filter_sym_id']
    symbols = data['symbols']

symbol_list = [sym.split('binance.')[1].split('_')[0] for sym in symbols]
for i in symbol_list:
    if i not in filter_sym_id.keys():
        print(i)

cap = {}
period = 200 # coinmarket cap拿数据如果period设置太少，会变成1小时周期。其次coinmarket cap的数据会做二次校正
date_update = str(datetime.now()).split(':')[0].replace('-','')+'08'
check_dt = int(datetime(datetime.now().year,datetime.now().month,datetime.now().day,8).timestamp()) # 这里如果机子是utc时间，要把8改成0
symbol_list = [sym.split('binance.')[1].split('_')[0] for sym in symbols]
base_url = 'https://api.coingecko.com/api/v3/coins/'

for sym in symbol_list[:4]:

    print(filter_sym_id[sym][0])   # 这里正常第一个是对的
    url = base_url + filter_sym_id[sym][0]+"/market_chart?vs_currency=usd&days="+str(period)
    print(url)
    session = Session()
    session.headers.update()
    market_cap_response = session.get(url)
    print(market_cap_response)
    cap_data = json.loads(market_cap_response.text)

    df =  pd.DataFrame(cap_data['market_caps'])

    df[0] = (df[0]//86400*86400/1000).astype(int)

    cap[sym] = df.set_index(0)
    time.sleep(4)# 防止被ban

cap_d = pd.DataFrame()
name_list = []
for i in cap.keys():
    name_list.append(i)
    cap_d = pd.concat([cap_d,cap[i]],axis=1)
    cap_d.columns = name_list


cap_d.rename(columns={'shib':'1000shib'},inplace=True)
list_c = []
for c in cap_d.columns:
    list_c.append('binance.'+c+'_usdt_swap')
cap_d.columns = list_c
cap_d = cap_d.loc[:check_dt].fillna(method='ffill')
index_array = np.array(cap_d.index)
index_array = index_array[index_array%86400==0]
cap_d = cap_d.loc[index_array,:].sort_index()

cap_d.to_hdf('market_cap.h5',key = 'mc')

bitcoin
https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=200
<Response [200]>
ethereum
https://api.coingecko.com/api/v3/coins/ethereum/market_chart?vs_currency=usd&days=200
<Response [200]>
binancecoin
https://api.coingecko.com/api/v3/coins/binancecoin/market_chart?vs_currency=usd&days=200
<Response [200]>
cardano
https://api.coingecko.com/api/v3/coins/cardano/market_chart?vs_currency=usd&days=200
<Response [200]>


In [9]:
cap_d

,binance.btc_usdt_swap,binance.eth_usdt_swap,binance.bnb_usdt_swap,binance.ada_usdt_swap
0,,,,
1673049600,3.265714e+11,1.531154e+11,4.240410e+10,9.774060e+09
1673136000,3.263717e+11,1.523518e+11,4.264799e+10,9.715965e+09
1673222400,3.284709e+11,1.548579e+11,4.465691e+10,1.030061e+10
1673308800,3.311582e+11,1.593975e+11,4.454201e+10,1.112233e+10
1673395200,3.357439e+11,1.609992e+11,4.531822e+10,1.129266e+10
...,...,...,...,...
1689897600,5.793087e+11,2.274843e+11,3.784336e+10,1.107721e+10
1689984000,5.813979e+11,2.274676e+11,3.798030e+10,1.097036e+10
1690070400,5.770685e+11,2.233088e+11,3.747238e+10,1.079705e+10
